In [ ]:
!pip install pyngrok
from pyngrok import ngrok
import locale
locale.getpreferredenoding = lambda: "UTF:8"


In [ ]:
# !export LC_ALL=en_US.UTF-8
# locale.getpreferredenoding = lambda: "ANSI_X3.4-1968"


In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/tloen/alpaca-lora.git

In [ ]:
%cd alpaca-lora

In [ ]:
!pip install -u pip
!pip install -r requirements.txt
!pip install torch==2.0.0
!pip install torch

In [ ]:
import torch
from peft import PeftModel
import transformers
import textwrap
from transformers import LlamaTokenizer, GenerationConfig, LlamaForCausalLM
from transformers.generation.utils import GreedySearchDecoderOnlyOutput

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

In [ ]:
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", load_in_8bit=True, device_map="auto")

In [ ]:
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b", torch_dtype=torch.float16)

In [ ]:
    # unwind broken decapoda-research config
    model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
    model.config.bos_token_id = 1
    model.config.eos_token_id = 2

In [ ]:
model = model.eval()
model = torch.compile(model)

In [ ]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request

### Instruction:
[Instruction]

### Response:
"""

def create_prompt(prompt):
  final_prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request

### Instruction:
[{prompt}]

### Response:
"""
  return final_prompt

In [ ]:
final_prompt = create_prompt("berikan rumus integral")

In [ ]:
encoding = tokenizer(final_prompt, return_tensors="pt" )

In [ ]:
input_ids = encoding["input_ids"].to(DEVICE)

In [ ]:
generation_config = GenerationConfig(temperature=0.1, top_p=0.75, repetition_penalty=1.1)

In [ ]:
with torch.inference_mode():
    output = model.generate(input_ids=input_ids, 
                            generation_config=generation_config, 
                            return_dict_in_generate=True, 
                            output_scores=True,
                            max_new_tokens=256)

In [ ]:
response = tokenizer.decode(output.sequences[0])

print(response.split("### Document Classs:")[-1].strip())

In [ ]:
token = input("auth token : ")
ngrok.set_auth_token(token)
# start ngrok tunnel
public_url = ngrok.connect(5000).public_url
print(public_url)

In [ ]:
# !export LC_ALL=en_US.UTF-8
# !pip install pyngrok

from flask import Flask, request, jsonify
import os
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/myendpoint', methods=['POST'])
def prompt():
    data = request.get_json()
    print("data:",data)
    prompt = create_prompt(data['prompt'])
    print("prompt:",prompt)
    encoding = tokenizer(prompt, return_tensors="pt" )
    input_ids = encoding["input_ids"].to(DEVICE)
    generation_config = GenerationConfig(temperature=0.1, top_p=0.75, repetition_penalty=1.1)
    with torch.inference_mode():
      output = model.generate(input_ids=input_ids, 
                              generation_config=generation_config, 
                              return_dict_in_generate=True, 
                              output_scores=True,
                              max_new_tokens=256)
    # Do something with the data here
    output2 = tokenizer.decode(output.sequences[0])
    response_index = output2.split("### Document Classs:")[-1].strip().find("### Response:")
    res = output2[response_index+len("### Response:"):].strip()

    response = {'response':res}

    print(response)
    return jsonify(response)

if __name__ == '__main__':
    port = 5000
    print('Public URL:', public_url)
    app.run(port=port)


In [ ]:
response = tokenizer.decode(output.sequences[0])

print(response.split("### Document Classs:")[-1].strip())

In [ ]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request

### Instruction:
[Instruction]

### Response:
"""

CLASSIFICATION_TEMPLATE = f"""
Below is the complete content of a document. Can you identify which category it is likely to be belonging to?.
The possible categories are given below.

1. Insurance claim request
2. Leave of absense letter 
3. Invitation letter

Write a response which identifies the document to any one of the above classes

### Content:

From,
Sherlock Holmes
221B Baker Street

Date: 01/04/2023

To,
General Insurance Company
Lombard Street London
United Kingdom

Dear sir/madam;

Re: Policy No - P-123-456-789

I am writing this letter in response to your letter on 30/03/2023, offering £400. to fix my claim. I don’t think that this is an appropriate sum.

I request you to check out my case. If I do not get a reasonable offer, I will have no alternative but to refer my case to the Financial Ombudsman Service.


I will review forward to get your offer within the following 14 days.

Yours sincerely/faithfully;

Sign and print your name.

### Document Classs:


"""

QUERY_TEMPLATE = f"""

Based on the tabular contents provided below, can you identify the total premium paid. Output the total premium paid only.

### Content:

### Response:

 """


def create_prompt(instruction: str) -> str:
    return PROMPT_TEMPLATE.replace("[Instruction]", instruction)




In [ ]:
prompt = CLASSIFICATION_TEMPLATE
encoding = tokenizer(prompt, return_tensors="pt" )


In [ ]:
input_ids = encoding["input_ids"].to(DEVICE)

In [ ]:
generation_config = GenerationConfig(temperature=0.1, top_p=0.75, repetition_penalty=1.1)

In [ ]:
with torch.inference_mode():
    output = model.generate(input_ids=input_ids, 
                            generation_config=generation_config, 
                            return_dict_in_generate=True, 
                            output_scores=True,
                            max_new_tokens=256)

In [ ]:
response = tokenizer.decode(output.sequences[0])

print(response.split("### Document Classs:")[-1].strip())